In [102]:
#含tag和tips
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

words = set(nltk.corpus.words.words())

c=pd.read_table('dataset_ubicomp2013\dataset_ubicomp2013_checkins.txt',sep='\t',header=None)
tag=pd.read_table('dataset_ubicomp2013\dataset_ubicomp2013_tags.txt',sep='\t',header=None,encoding = 'utf-8')
tip=pd.read_table('dataset_ubicomp2013\dataset_ubicomp2013_tips_1.txt',sep='\t',header=None,error_bad_lines=False)

group=c.groupby([0])
tip_group=tip.groupby([0])
#print(group.get_group(6))

user=[]
m_data=[]

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


def tran_tense_and_rm_stopwords(sentence):
    tokens = word_tokenize(sentence)  # 分词
    tagged_sent = pos_tag(tokens)     # 获取单词词性

    wnl = WordNetLemmatizer()
    lemmas_sent = []
    sent_without_stopword=[]
    for tag in tagged_sent:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) # 词形还原
    for word in lemmas_sent:
        if word not in stopwords.words('english'):
            sent_without_stopword.append(word)
    return str(sent_without_stopword)

def rm_not_english(sent):
    strs=''
    for w in nltk.wordpunct_tokenize(sent):
        if w.lower() in words:
            a="".join(w)
            strs=strs+a+','
    return strs

def rm_not_noun(sentence):
    strs=''
    tokens = nltk.word_tokenize(sentence.lower())
    tags = nltk.pos_tag(tokens)
    for word,pos in tags:
        if(pos=='NN'):
            a=''.join(word)
            strs=strs+a+','
    return strs

for _, g in group:
    ulist = g[0].tolist()
    user.append(list(set(ulist))[0])
for userid in user:
    res=group.get_group(userid)[1]
    tmp_seq = ""
    posId=0
    for resid in res:
        res_inf = str(tag[tag.values == resid][1].tolist())[2:-2].lower()
        res_inf=tran_tense_and_rm_stopwords(res_inf)
        res_inf=rm_not_english(res_inf)[0:-1]
        if(posId<len(res)-1):
            tmp_seq = tmp_seq + str(resid)+"_"+str(int(posId*(posId+1)/2))+"_"+res_inf+"#"
        else:
            tmp_seq = tmp_seq + str(resid)+"_"+str(int(posId*(posId+1)/2))+"_"+res_inf
        posId +=1
    tip_inf = str(tip_group.get_group(userid)[2].tolist())[2:-2].lower()
    tip_inf=tran_tense_and_rm_stopwords(tip_inf)
    tip_inf=rm_not_english(tip_inf)
    tip_inf=rm_not_noun(tip_inf)[0:-1]
    new_session_pd = {}
    new_session_pd['userId'] = userid
    new_session_pd['sequence'] = tmp_seq
    new_session_pd['tips'] = tip_inf
    m_data.append(dict(new_session_pd))
m_data_pd = pd.DataFrame(data=m_data)
print(m_data_pd[100:150])


     userId                                           sequence  \
100    2379  34938_0_baguette,bakery,bread,breakfast,brunch...   
101    2442  16119_0_#29910_1_late,night,pizza,olive,oil,cr...   
102    2443  15743_0_baguette,baker,s,dozen,bakery,bread,co...   
103    2448  75007_0_beer,coffee,coffee,roaster,counter,sea...   
104    2501  4156_0_bread,bakery,brownie,coffee,cup,cake,ev...   
105    2513                                     26194_0_coffee   
106    2538  144620_0_coffee,coffee#144620_1_coffee,coffee#...   
107    2591                 102278_0_food,tasty#46179_1_coffee   
108    2605  9262_0_cheese,cupcake,hot,chocolate#7455_1_foo...   
109    2621  121133_0_#16564_1_beautiful,space,chef,flatiro...   
110    2637  7680_0_coffee#7680_1_coffee#925_3_diner,kosher...   
111    2709  112711_0_baguette,bakery,bread,breakfast,brunc...   
112    2720  1411_0_bridge,height,outdoor,seating,restauran...   
113    2772  4843_0_restaurant,like,category,indicate,thoug...   
114    281

In [103]:
m_data_pd

,userId,sequence,tips
0,6,"278_0_lentil,soup,west,village#278_1_lentil,so...",
1,15,"1254_0_joe,wine#12508_1_mi,choice,eats,nam,goi...","peanut,butter,use,love,place,coffee,s,downhill..."
2,25,"16625_0_restaurant#72623_1_#47540_3_baguette,b...","check,order,screw,something,time,velvet,dark,s..."
3,72,"12222_0_beer,burrito,soho,yucca,fry#73254_1_fr...","el,mirador,morning,err,afternoon,wake,night,ch..."
4,99,"10786_0_hotel,restaurant#1085_1_#16699_3_brunc...","holy,chicken,place,sand,floor,thing,put,mouth,..."
...,...,...,...
2055,265904,"140749_0_bakery,best,bakery,world,best,coffee#...","pizza,slice,pope,selection,everything,beat,line"
2056,265965,"200_0_bar,crayon,happy,hour,restaurant#51660_1...","handicap,seat,area,floor"
2057,266532,"19182_0_restaurant,wine#22768_1_food,food#1918...","try,pineapple,fry,rice,jasmine,tea"
2058,266576,"10289_0_pizza,pizzeria#34553_1_coffee,rat,rate...","honor,check,coupon,deal"


In [104]:
m_data_pd[["userId","tips"]]

,userId,tips
0,6,
1,15,"peanut,butter,use,love,place,coffee,s,downhill..."
2,25,"check,order,screw,something,time,velvet,dark,s..."
3,72,"el,mirador,morning,err,afternoon,wake,night,ch..."
4,99,"holy,chicken,place,sand,floor,thing,put,mouth,..."
...,...,...
2055,265904,"pizza,slice,pope,selection,everything,beat,line"
2056,265965,"handicap,seat,area,floor"
2057,266532,"try,pineapple,fry,rice,jasmine,tea"
2058,266576,"honor,check,coupon,deal"


In [119]:
txtfile = 'dataset_ubicomp2013\dataset_ubicomp2013_tags.txt'
m_data_dict = {}
with open(txtfile, 'r', encoding='utf-8') as infile:
    stripped = (line.strip() for line in infile)
    for line in stripped:
        if line:
            #outfile.write(" ".join(alpha for alpha in line.split(",") if alpha.isalpha() or alpha.isalnum()))
            line_tmp = " ".join(line.split(",")).split("\t")
            new_session_pd = {}
            new_session_pd['ItemId'] = line_tmp[0]
            if len(line_tmp) == 1:
                new_session_pd['tags'] = ""
                m_data_dict[line_tmp[0]] = ""
            else:
                new_session_pd['tags'] = line_tmp[1]
                m_data_dict[line_tmp[0]] = line_tmp[1]

In [122]:
tags_info_text = [i for i in m_data_dict.values()]

In [168]:
tips_info_text = [" ".join(i.split(",")) for i in m_data_pd["tips"].tolist()]

In [130]:
total_info = np.concatenate((tags_info_text, tips_info_text))

In [169]:
print(len(total_info))
print(len(tags_info_text))
print(len(tips_info_text))

5358
3298
2060


In [171]:
tags_info_text

['',
 "andy cohen bakery bar barbeque bbq bistro boutique bravo bravoandy celebrity sighting coffee cupcake dessert desserts douchebag douchebags foodie foody french cuisine gallery art gossip girl gossipgirl home manhattan models people hot spot photo booth restaurant socalite socialite steak frites theater top chef tourists trendy victoria's secret - bombshell vs hotspot west village wine zagat rated",
 'colombian cupcake fish tacos grilled corn long wait pork rice and beans sweet plantains tinituan ma lala trendy',
 'brunch happy hour irish pub',
 'french soho zagat-rated zagats',
 'byob',
 '+ jazz club good food and jazz',
 'bar beer brunch burger cozy dinner group happy hour lunch nice waitress salad sandwiches sports steak tavern',
 'bakery bravo cupcake dessert diner frozen hot chocolate frozen yogurt gossip girl ice cream socialite top chef tourists upper east side',
 'bagels cafe coffee desserts green tea cookies irving farm local food outdoor seating tea wifi',
 'gossip girl'

In [170]:
tips_info_text

['',
 'peanut butter use love place coffee s downhill try',
 'check order screw something time velvet dark stormy float diner food dim light place item fall mushroom lack variety flavor fry sit pool oil',
 'el mirador morning err afternoon wake night check lot talk date spot sit corner bar view room absolute t pig part part crispy part pizza s need spring st lafayette mulberry york p hidden room year realize lunch mid day meeting tea sit front summer place area s watch s restaurant pho combination rice soup pork chop need lazy lunch degree place today need place hood day re feel choice dog date spot eat home cook food cow eat farm sit window bench bar watch eat lalo m talk dessert dinner place hood eat place move love lunch ball thing sit bench t anything experience seaweed salad deliciousness morning meeting bring something read party snag half circle booth wall spot ginger citrus tea decompress honey thing world bourbon honey sister s visit slam random thing menu cheese ravioli hey n

In [172]:
from tensorflow.contrib import learn
vocab = learn.preprocessing.VocabularyProcessor(30)

In [173]:
import numpy as np
text = np.array(list(vocab.fit_transform([i for i in total_info])))
num_vocab = len(vocab.vocabulary_)
num_nodes = len(text)

In [174]:
text_i = 0
tag_m_data = []
for i,key in enumerate(m_data_dict.keys()):
    new_session_pd = {}
    new_session_pd['ItemId'] = key
    new_session_pd['tags'] = " ".join([str(j) for j in text[text_i]])
    tag_m_data.append(dict(new_session_pd))
    text_i += 1
    
import pandas as pd
import pickle

def save_obj(obj):
    with open('./datasets/foursquare_seq_tags.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

tag_m_data_pd = pd.DataFrame(data=tag_m_data)
save_obj(tag_m_data_pd)

user_tips_data = {}
for row in m_data_pd[["userId","tips"]].itertuples():
    user_tips_data[str(row.userId)] = " ".join([str(j) for j in text[text_i]])
    text_i += 1

In [186]:
tag_m_data_pd["tags"][2155]

'254 1135 13 89 672 85 2795 136 13 136 110 1136 91 588 36 95 338 1726 110 0 0 0 0 0 0 0 0 0 0 0'

In [176]:
total_info

array(['',
       "andy cohen bakery bar barbeque bbq bistro boutique bravo bravoandy celebrity sighting coffee cupcake dessert desserts douchebag douchebags foodie foody french cuisine gallery art gossip girl gossipgirl home manhattan models people hot spot photo booth restaurant socalite socialite steak frites theater top chef tourists trendy victoria's secret - bombshell vs hotspot west village wine zagat rated",
       'colombian cupcake fish tacos grilled corn long wait pork rice and beans sweet plantains tinituan ma lala trendy',
       ..., 'try,pineapple,fry,rice,jasmine,tea',
       'honor,check,coupon,deal', 'grill,cheese,tomato,soup,fail'],
      dtype='<U13705')

In [177]:
text

array([[   0,    0,    0, ...,    0,    0,    0],
       [   1,    2,    3, ...,   28,   29,   30],
       [  57,   14,   58, ...,    0,    0,    0],
       ...,
       [1367, 3603, 3466, ...,    0,    0,    0],
       [3970, 1212, 3837, ...,    0,    0,    0],
       [ 802,  224, 2283, ...,    0,    0,    0]], dtype=int64)

In [178]:
import pandas as pd
import pickle

c=pd.read_table("./dataset_ubicomp2013/dataset_ubicomp2013_checkins.txt",sep='\t',header=None)
tag=pd.read_table('./dataset_ubicomp2013/dataset_ubicomp2013_tags.txt',sep='\t',header=None,encoding = 'utf-8')
tip=pd.read_table('./dataset_ubicomp2013/dataset_ubicomp2013_tips_1.txt',sep='\t',header=None,error_bad_lines=False)

group=c.groupby([0])
tip_group=tip.groupby([0])
#print(group.get_group(6))

user=[]
m_data=[]

for _, g in group:
    ulist = g[0].tolist()
    user.append(list(set(ulist))[0])
for userid in user:
    res=group.get_group(userid)[1]
    tmp_seq = ""
    posId=0
    for resid in res:
        res_inf=''
        if(posId<len(res)-1):
            tmp_seq = tmp_seq + str(resid)+"_"+str(int(posId*(posId+1)/2))+"#"
        else:
            tmp_seq = tmp_seq + str(resid)+"_"+str(int(posId*(posId+1)/2))
        posId +=1
    #tip_inf=''
    new_session_pd = {}
    new_session_pd['SessionId'] = userid
    new_session_pd['Sequence'] = tmp_seq
    new_session_pd['tips'] = user_tips_data[str(userid)]
    m_data.append(dict(new_session_pd))
user_tips_data_pd = pd.DataFrame(data=m_data)
print(user_tips_data_pd[100:150])
def save_obj(obj):
    with open('./datasets/foursquare_seq.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj(user_tips_data_pd)

     SessionId                                           Sequence  \
100       2379                                            34938_0   
101       2442  16119_0#29910_1#47569_3#28614_6#1130_10#16377_...   
102       2443                    15743_0#41127_1#15743_3#41127_6   
103       2448  75007_0#34591_1#11114_3#75007_6#75007_10#34591...   
104       2501  4156_0#47056_1#110119_3#4156_6#4156_10#16189_1...   
105       2513                                            26194_0   
106       2538  144620_0#144620_1#144620_3#144620_6#144620_10#...   
107       2591                                   102278_0#46179_1   
108       2605  9262_0#7455_1#41961_3#15608_6#184152_10#8713_1...   
109       2621  121133_0#16564_1#20001_3#164_6#8041_10#38670_1...   
110       2637  7680_0#7680_1#925_3#16126_6#7680_10#102886_15#...   
111       2709  112711_0#51660_1#51152_3#1411_6#112711_10#5166...   
112       2720                      1411_0#1411_1#52205_3#52205_6   
113       2772  4843_0#5705_1#2236

In [187]:
user_tips_data_pd

,SessionId,Sequence,tips
0,6,278_0#278_1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,15,1254_0#12508_1#11660_3#910_6#1329_10#1254_15#1...,1098 992 3455 523 847 13 3456 3457 1367 0 0 0 ...
2,25,16625_0#72623_1#47540_3,1212 274 3458 3459 467 717 404 3460 3461 98 85...
3,72,12222_0#73254_1#5004_3#4283_6#12222_10#73254_1...,3467 3468 3469 3470 3471 3472 187 1212 3348 34...
4,99,10786_0#1085_1#16699_3#577_6#1316_10#4529_15,3644 182 847 1275 3645 2984 3646 2625 3647 85 ...
...,...,...,...
2055,265904,140749_0#140749_1#140749_3#382_6#140749_10#140...,441 1595 6025 1338 3624 3548 829 0 0 0 0 0 0 0...
2056,265965,200_0#51660_1#5867_3#1073_6#667293_10#6305_15#...,6026 3524 543 3645 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
2057,266532,19182_0#22768_1#19182_3#47026_6#47026_10#4310_...,1367 3603 3466 65 2240 110 0 0 0 0 0 0 0 0 0 0...
2058,266576,10289_0#34553_1#17779_3,3970 1212 3837 3722 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
